# Pull MIP table files; Create CVs
<div style="text-align: right">
<p>
    <img src="https://pcmdi.github.io/assets/PCMDI/199x65px-PCMDI-Logo-Text-rectangle.png"
         width="91"
         height="30"
         class="fixed-height-image"
         style="margin-right: 20px"
         title="Program for Climate Model Diagnosis and Intercomparison"
         alt="Program for Climate Model Diagnosis and Intercomparison"
    >&nbsp;
    <img src="https://pcmdi.github.io/assets/LLNL/212px-LLNLiconPMS286-WHITEBACKGROUND.png"
         width="30"
         height="30"
         class="fixed-height-image"
         style="margin-right: 20px"
         title="Lawrence Livermore National Laboratory"
         alt="Lawrence Livermore National Laboratory"
    >&nbsp;
    <img src="https://pcmdi.github.io/assets/DOE/459x127px-DOE-Logo_Color_TextOnly.png"
         width="108"
         height="30"
         class="fixed-height-image"
         style="margin-right: 20px"
         title="United States Department of Energy"
         alt="United States Department of Energy"
    >
</p>
</div>

**Summary**

This file pulls input4MIPs CMIP6Plus/CMOR3.7.x-era MIP table files, strips out extraneous variables and saves the files for local use. It also creates a bare-minimum set of input4MIPs Controlled Vocabulary files to meet the needs of the PCMDI-AMIP-1-1-10 data generation.

**Authors**

Paul J. Durack ([durack1](https://github.com/durack1); [PCMDI](https://pcmdi.llnl.gov/), [Lawrence Livermore National Laboratory](https://www.llnl.gov/))

**Notes**

PJD 14 Feb 2025 - initiated<br>

TODO:

**Links**

### imports

In [1]:
%%time
import datetime
import json
import os
import requests

CPU times: user 40 ms, sys: 14.3 ms, total: 54.3 ms
Wall time: 60.1 ms


### set table files and pull

In [2]:
%%time
urls = {
    "coordinate": "https://raw.githubusercontent.com/PCMDI/input4mips-cmor-tables/refs/heads/master/Tables/input4MIPs_coordinate.json",
    "formula_terms": "https://raw.githubusercontent.com/PCMDI/input4mips-cmor-tables/refs/heads/master/Tables/input4MIPs_formula_terms.json",
    "Ofx": "https://raw.githubusercontent.com/PCMDI/input4MIPs-cmor-tables/refs/heads/master/Tables/input4MIPs_Ofx.json",
    "Omon": "https://raw.githubusercontent.com/PCMDI/input4MIPs-cmor-tables/refs/heads/master/Tables/input4MIPs_Omon.json",
    "SImon": "https://raw.githubusercontent.com/PCMDI/input4MIPs-cmor-tables/refs/heads/master/Tables/input4MIPs_SImon.json",
}

# loop through urls
keys = urls.keys()
for count, key in enumerate(keys):
    print(count, key)
    url = urls[key]
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4/5xx)
        vars()[key] = json.loads(response.text)
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON decode failed: {e}")
    except Exception as e:
        print(f"Unexpected error occurred: {e}")

0 coordinate
1 formula_terms
2 Ofx
3 Omon
4 SImon
CPU times: user 11.2 ms, sys: 5.02 ms, total: 16.2 ms
Wall time: 83.8 ms


### all tables trim out redundant variables, reset table_date

In [3]:
%%time
# Ofx
keyList = list(Ofx["variable_entry"].keys())
keepKeys = ["areacello", "sftof"]
for count, key in enumerate(keyList):
    if key not in keepKeys:
        Ofx["variable_entry"].pop(key)
# Omon
keyList = list(Omon["variable_entry"].keys())
keepKeys = ["tos", "tosbcs"]
for count, key in enumerate(keyList):
    if key not in keepKeys:
        Omon["variable_entry"].pop(key)
# SImon - no redundant variables
keyList = list(SImon["variable_entry"].keys())
keepKeys = ["siconc", "siconcbcs"]
for count, key in enumerate(keyList):
    if key not in keepKeys:
        Omon["variable_entry"].pop(key)
Header = SImon["Header"]
Header

CPU times: user 22 μs, sys: 1 μs, total: 23 μs
Wall time: 24.8 μs


{'Conventions': 'CF-1.7 CMIP-6.2',
 'approx_interval': '30.00000',
 'cmor_version': '3.5',
 'data_specs_version': '01.00.33',
 'generic_levels': '',
 'int_missing_value': '-999',
 'mip_era': 'CMIP6',
 'missing_value': '1e20',
 'product': 'input4MIPs',
 'realm': 'seaIce',
 'table_date': '29 May 2024',
 'table_id': 'Table input4MIPs_SImon'}

### cleanup all Table file Headers

In [4]:
Header["cmor_version"] = "3.11"
Header["mip_era"] = "CMIP7"
Header["table_date"] = datetime.datetime.now().strftime("%Y-%m-%d")

### write all files out to Tables subdir

In [5]:
%%time
files = ["coordinate", "formula_terms", "Ofx", "Omon", "SImon"]
for count, name in enumerate(files):
    print(count, name)
    dic = eval(name)
    # add Header
    if name not in ["coordinate", "formula_terms"]:
        # exclude Header rewrite for coordinate/formula_terms as changes CMOR 3.9.0 behaviours
        dic["Header"] = Header
        dic["Header"]["table_id"] = "_".join(["Table input4MIPs", name])
    # write file
    outFile = "".join(["../Tables/input4MIPs_", name, ".json"])
    with open(outFile, "w") as f:
        json.dump(
            dic, f, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":")
        )

0 coordinate
1 formula_terms
2 Ofx
3 Omon
4 SImon
CPU times: user 2.8 ms, sys: 1.76 ms, total: 4.56 ms
Wall time: 3.6 ms
